参考: https://juejin.cn/post/7341210909068574760

首先安装依赖
```
%pip install jupyter_contrib_nbextensions
%pip install llama-index-core
%pip install llama-index-llms-openai
%pip install llama-index-llms-replicate
%pip install llama-index-embeddings-huggingface
!TMPDIR=/home/pengfei.xiong/tmp pip install --cache-dir=$TMPDIR llama-index-embeddings-huggingface
%pip install llama-index-llms-ollama
%pip install llama-index
%pip install llama-index-embeddings-ollama
%pip install llama-index-vector-stores-postgres
```

In [8]:
# chat 测试
from llama_index.llms.ollama import Ollama
llm = Ollama(model="qwen2:7b", request_timeout=300, base_url='http://10.24.20.73:11434')
resp = llm.complete("你是谁?")
print(resp)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/usr/lib/ssl/certs/ca-certificates.crt'
load_verify_locations cafile='/usr/lib/ssl/certs/ca-certificates.crt'
DEBUG:httpcore.connection:connect_tcp.started host='10.24.20.73' port=11434 local_address=None timeout=300.0 socket_options=None
connect_tcp.started host='10.24.20.73' port=11434 local_address=None timeout=300.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f1dababef50>
connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f1dababef50>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
send_request_headers.complete
DEBUG:httpcore.http11:send_request_b

In [6]:
# embedding测试
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="gte-Qwen2-1.5B-instruct:latest",
    base_url='http://10.24.20.73:11434',
    ollama_additional_kwargs={"mirostat": 0},
)

pass_embedding = ollama_embedding.get_text_embedding_batch(
    ["This is a passage!", "This is another passage"], show_progress=True
)
print("pass_embedding:")
print(pass_embedding)
print("query_embedding:")
query_embedding = ollama_embedding.get_query_embedding("Where is blue?")
print(query_embedding)
print(len(query_embedding))
embedding_dim = len(query_embedding)

/home/pengfei.xiong/jupyterProjects/medical_rag/medical-rag-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None


Generating embeddings: 100%|██████████| 2/2 [00:04<00:00,  2.04s/it]

pass_embedding:
[[-7.374053001403809, 6.589729309082031, 5.327824115753174, -0.6322519779205322, 1.2912912368774414, -2.545228958129883, 2.7866973876953125, 1.5356662273406982, -5.5174970626831055, 0.9315221309661865, -0.5998791456222534, 3.505786180496216, -1.8053842782974243, 3.307161331176758, -1.0759779214859009, -2.3927671909332275, -0.8681997656822205, -0.8634998798370361, -1.1080083847045898, -1.732651710510254, 0.9145500659942627, -6.372185707092285, -2.12833309173584, -0.585984468460083, -3.591080904006958, -5.9416823387146, -3.217790126800537, 0.14151568710803986, 0.1990167796611786, 2.648514747619629, -1.3350955247879028, -1.9101125001907349, 4.566057205200195, 15.288345336914062, 5.104739189147949, -0.1259223073720932, -3.407099962234497, -0.8688140511512756, 2.7257370948791504, -1.3404920101165771, -0.2792310118675232, 3.3443689346313477, 0.9550212621688843, 0.15120546519756317, 19.998579025268555, -0.22424231469631195, 0.6693404316902161, -4.574716091156006, -1.4966602325

In [11]:
# 下载测试数据
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

1130.65s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1135.81s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


--2024-07-23 16:45:05--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.1s    

2024-07-23 16:45:06 (611 KB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [3]:
import logging
import sys

# Uncomment to see debug logs
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
import textwrap
import openai

In [9]:
# 加载测试数据
documents = SimpleDirectoryReader("./data/paul_graham").load_data()
print("Document ID:", documents[0].doc_id)

DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 1
> [SimpleDirectoryReader] Total files added: 1
DEBUG:fsspec.local:open file: /home/pengfei.xiong/jupyterProjects/medical_rag/data/paul_graham/paul_graham_essay.txt
open file: /home/pengfei.xiong/jupyterProjects/medical_rag/data/paul_graham/paul_graham_essay.txt
Document ID: 2685670b-0f52-4d2e-8dbb-a896031d6320


In [1]:
# 创建数据库
# 这里用parade db
import psycopg2

db_name = "dify2"
host = '127.0.0.1'
port = '5435'
user = 'postgres'
password = 'postgres'
conn = psycopg2.connect(dbname=db_name, user=user,
        password=password, host=host, port=port)
conn.autocommit = True

# with conn.cursor() as c:
#     c.execute(f"CREATE DATABASE vector_db")

In [13]:
# embeding
from llama_index.core import Settings
Settings.embed_model = ollama_embedding
vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="paul_graham_essay",
    embed_dim=embedding_dim,  # openai embedding dimension
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, show_progress=True
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: What I Worked On

February 2021

Before college...
> Adding chunk: What I Worked On

February 2021

Before college...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: All that seemed left for philosophy were edge c...
> Adding chunk: All that seemed left for philosophy were edge c...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Its brokenness did, as so often happens, genera...
> Adding chunk: Its brokenness did, as so often happens, genera...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: If he even knew about the strange classes I was...
> Adding chunk: If he even knew about the strange classes I was...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The students and faculty in the painting depart...
> Adding chunk: The students and faculty in the painting depart...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: I wanted to go back to RISD, but I was

Generating embeddings:   0%|          | 0/22 [00:00<?, ?it/s]

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.

Generating embeddings:  45%|████▌     | 10/22 [00:04<00:05,  2.26it/s]

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.

Generating embeddings:  91%|█████████ | 20/22 [00:08<00:00,  2.36it/s]

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None


Generating embeddings: 100%|██████████| 22/22 [00:09<00:00,  2.39it/s]


In [16]:
query_engine = index.as_query_engine(llm=llm)
# ask something
response = query_engine.query("What did the author do?")
print(textwrap.fill(str(response), 100))


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node e7d1c38b-ae6f-4cac-acae-c6acb62c54a6] [Similarity score:             0.33894] What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...
> [Node 24636881-0d1a-4010-977d-16a5702cfe5b] [Similarity score:             0.33894] What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...
> Top 2 nodes:
> [Node e7d1c38b-ae6f-4cac-acae-c6acb62c54a6] [Similarity score:             0.33894] What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...
> [Node 24636881-0d1a-4010-977d-16a5702cfe5b] [Similarity score:             0.3

In [17]:
response = query_engine.query("What happened in the mid 1980s?")
print(textwrap.fill(str(response), 100))


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.24.20.73:11434
Starting new HTTP connection (1): 10.24.20.73:11434
DEBUG:urllib3.connectionpool:http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
http://10.24.20.73:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node e7d1c38b-ae6f-4cac-acae-c6acb62c54a6] [Similarity score:             0.328227] What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...
> [Node 24636881-0d1a-4010-977d-16a5702cfe5b] [Similarity score:             0.328227] What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...
> Top 2 nodes:
> [Node e7d1c38b-ae6f-4cac-acae-c6acb62c54a6] [Similarity score:             0.328227] What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...
> [Node 24636881-0d1a-4010-977d-16a5702cfe5b] [Similarity score:             